In [ ]:
import webbrowser

# URL de LinkedIn
url = 'https://www.linkedin.com'

# Abre la URL en el navegador predeterminado
webbrowser.open(url)


In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pickle  # Para guardar y cargar cookies

# Credenciales de inicio de sesión
email = 'Gbustosgarcia01@gmail.com'
password = 'LG030920'

# Configuración del WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecuta el navegador en modo headless (sin GUI)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# Inicializa el navegador usando webdriver-manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# URL de LinkedIn
login_url = 'https://www.linkedin.com/login'

# Abre la página de inicio de sesión
driver.get(login_url)

# Encuentra los campos de email y contraseña y el botón de inicio de sesión
email_field = driver.find_element(By.ID, 'username')
password_field = driver.find_element(By.ID, 'password')
login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')

# Ingresa las credenciales
email_field.send_keys(email)
password_field.send_keys(password)

# Envía el formulario
login_button.click()

# Espera para asegurar que el inicio de sesión se complete
time.sleep(10)

# Guarda las cookies en un archivo
cookies = driver.get_cookies()
with open('linkedin_cookies.pkl', 'wb') as file:
    pickle.dump(cookies, file)

print("Cookies guardadas exitosamente.")

# Cierra el navegador
driver.quit()


Cookies guardadas exitosamente.


In [16]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pickle

# Configuración del WebDriver
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Ejecuta el navegador en modo headless
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("--disable-dev-shm-usage")  # Usa /dev/shm en lugar de /tmp

# Inicializa el navegador usando webdriver-manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Carga las cookies desde el archivo
with open('linkedin_cookies.pkl', 'rb') as file:
    cookies = pickle.load(file)
    driver.get('https://www.linkedin.com/')
    for cookie in cookies:
        driver.add_cookie(cookie)
    driver.refresh()

# Cargar el archivo CSV
df = pd.read_csv('linkedin_profiles.csv')

# Lista para almacenar los resultados
results = []

# Iterar sobre cada fila del DataFrame
for index, row in df.iterrows():
    search_url = row['LinkedIn URL perfiles']
    driver.get(search_url)
    
    # Espera para asegurar que la página cargue completamente
    time.sleep(5)
    
    # Extrae los enlaces de perfil en la primera página
    profile_links = driver.find_elements(By.XPATH, '//a[contains(@href, "linkedin.com/in/")]')
    
    seen_links = set()
    
    if not profile_links:
        results.append({
            'Name': row['Name'],
            'Service Company': row['Service Company'],
            'LinkedIn Profile URL': 'No results found'
        })
    else:
        for link in profile_links:
            href = link.get_attribute('href')
            if href not in seen_links:
                seen_links.add(href)
                results.append({
                    'Name': row['Name'],
                    'Service Company': row['Service Company'],
                    'LinkedIn Profile URL': href
                })
                
                # Escribir la nueva fila en el archivo CSV inmediatamente
                pd.DataFrame([{
                    'Name': row['Name'],
                    'Service Company': row['Service Company'],
                    'LinkedIn Profile URL': href
                }]).to_csv('profiles.csv', mode='a', header=False, index=False)

# Cierra el navegador
driver.quit()
